# 第一步：拉大神的项目，准备环境

备注：我们的测试环境是colab的a100GPU环境，所用参数大概需要消耗37GB显存，如果显存不够可以自己改一下超参数，等下会说

In [ ]:
!git clone https://github.com/ssbuild/chatglm_finetuning

把代码拷贝到工作目录

In [ ]:
# 拷贝到colab目录，如果是自己的环境就拷贝到自己的目录
cp -r /content/chatglm_finetuning/* /content/

In [ ]:
# 安装依赖包
!pip install -U deep_training cpm_kernels icetk transformers>=4.26.1 deepspeed

# 第二部：制作数据集
使用我们的merge.py 选择对应的转换文件夹，脚本会自动将文件夹下面的所有alpaca格式的json文件转换成chatglm_finetuning所需的特殊json格式，输出为output.json

In [ ]:
!python merge.py

: 

# 把数据集挪到data里面
mv ./output.json ./data

# 修改train_util.py的超参数
我们的参数如下：请根据自己实际情况自行改动
```
train_info_args = {
    'devices': 1,
    'data_backend': 'record',
    'model_type': 'chatglm',
    # 预训练模型路径 , 因为用的是colab，直接从hunggingface拉是很快的
    'model_name_or_path': 'THUDM/chatglm-6b',
    'config_name': './config/config.json',
    'tokenizer_name': 'THUDM/chatglm-6b',
    'convert_onnx': False, # 转换onnx模型
    'do_train': True,
    'train_file':  [ './data/output.json'],
    'max_epochs': 38,
    'max_steps': -1,
    'optimizer': 'lion', # one of adamw,adam,lamb,lion
    'scheduler_type': 'linear',
    'optimizer_betas': (0.9, 0.999),
    'train_batch_size': 4,
    'eval_batch_size': 2,
    'test_batch_size': 2,
    'learning_rate': 2e-5,  #
    'adam_epsilon': 1e-8,
    'gradient_accumulation_steps': 1,
    'max_grad_norm': 1.0,
    'weight_decay': 0,
    'warmup_steps': 0,
    'output_dir': './output',
    'max_seq_length': 2048, # 如果资源充足，推荐长度2048 与官方保持一致
    'max_target_length': 100,  # 预测最大长度, 保留字段
    'use_fast_tokenizer': False,
    'do_lower_case': False,

    ##############  lora模块
    'with_lora': True,  # 是否启用lora模块，这里必须要启动，不然小数据集下根本无法训练出好的效果
    'inference_mode': False, # 推理模型, 不需要手动设置
    'r': 32, # 有多好的卡，就设置多大把
    'target_modules': ['dense','dense_h_to_4h','dense_4h_to_h','query_key_value'], # 尽量能lora的层都lora一下
    'target_dtype': '16',
    'lora_alpha': 32,
    # 'enable_lora': [True],
    'enable_lora': None,
    'lora_dropout': 0.1,
    'bias': 'none',  # Bias type for Lora. Can be 'none', 'all' or 'lora_only'"
}
```

In [ ]:
# 开启数据预处理，请记得如果数据集有更换请删除临时文件
!python data_utils.py

In [ ]:
# 模型训练，我研究过N多方案。包括自己写，能收敛有效果的只有这个
!python train.py

# 测试一下吧！
这里推荐直接在notebook加载模型，因为lora加载慢得抠脚

In [ ]:
import os
import torch
from deep_training.data_helper import ModelArguments, TrainingArguments, DataArguments
from deep_training.nlp.models.chatglm import TransformerChatGlmLMHeadModel, setup_model_profile, ChatGLMConfig,ChatGLMForConditionalGeneration
from deep_training.nlp.models.lora import LoraArguments, LoraModel
from transformers import HfArgumentParser

from data_utils import train_info_args, NN_DataHelper,get_deepspeed_config
from tokenization_chatglm import ChatGLMTokenizer


class MyTransformer(TransformerChatGlmLMHeadModel, with_pl=True):
    def __init__(self, *args, **kwargs):
        lora_args: LoraArguments = kwargs.pop('lora_args')
        super(MyTransformer, self).__init__(*args, **kwargs)
        self.lora_args = lora_args
        if lora_args.with_lora:
            model = LoraModel(self.backbone, lora_args)
            print('*' * 30,'lora info')
            model.print_trainable_parameters()
            self.set_model(model, copy_attr=False)



if __name__ == '__main__':
    train_info_args['seed'] = None
    parser = HfArgumentParser((ModelArguments, TrainingArguments, DataArguments, LoraArguments))
    model_args, training_args, data_args, _ = parser.parse_dict(train_info_args)

    setup_model_profile()

    dataHelper = NN_DataHelper(model_args, training_args, data_args)
    tokenizer: ChatGLMTokenizer
    tokenizer, _, _, _ = dataHelper.load_tokenizer_and_config(
        tokenizer_class_name=ChatGLMTokenizer, config_class_name=ChatGLMConfig)


    config = ChatGLMConfig.from_pretrained('./best_ckpt')
    config.initializer_weight = False

    lora_args = LoraArguments.from_pretrained('./best_ckpt')

    assert lora_args.inference_mode == True

    model = MyTransformer(config=config, model_args=model_args, training_args=training_args,lora_args=lora_args)
    # 加载lora权重
    model.backbone.from_pretrained(model.backbone.model, pretrained_model_name_or_path = './best_ckpt', lora_config = lora_args)

    base_model: ChatGLMForConditionalGeneration = model.backbone.model.model
    # 按需修改
    base_model.half().cuda()
    base_model = base_model.eval()

# 实验结论
1、loss降得很低是ok的，基本不会影响模型原来的能力

2、数据集小epoch可以设置得多一些，也不会影响模型能力

3、如果不使用lora，则基本等于从0开始训练模型了

In [ ]:
# 数据集内问题完美回答
response, history = base_model.chat(tokenizer, "什么是精益?", history=[],max_length=512,
                                        eos_token_id=config.eos_token_id,
                                        do_sample=True, top_p=0.7, temperature=0.95,)
print(response)

In [ ]:
# 写方案能力和结构化能力突然变强
response, history = base_model.chat(tokenizer, "帮我写一个团支部的共建方案", history=[],max_length=512,
                                        eos_token_id=config.eos_token_id,
                                        do_sample=True, top_p=0.7, temperature=0.95,)
print(response)

In [ ]:
# 翻译能力几乎丧失
response, history = base_model.chat(tokenizer, "翻译：nice to see you", history=[],max_length=512,
                                        eos_token_id=config.eos_token_id,
                                        do_sample=True, top_p=0.7, temperature=0.95,)
print(response)